In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import cv2

rad2deg = 180 / np.pi

In [ ]:
%matplotlib inline

In [ ]:
path = '/scratch/ssd/cciw/sample_data/'
images = os.listdir(path)
#img = images[-3]
img = '1340_2016-07-11_1_GLN_3447.JPG'
#img = '1349_2016-07-06_2_GLN_3061.JPG'
#img = '1342_2016-07-07_2_GLN_3182.JPG'
#img = 'sudoku.png'

In [ ]:
f = os.path.join(path, img)

In [ ]:
im = cv2.imread(f)

In [ ]:
im.shape  # (height, width)

In [ ]:
ds = 8  # naive downsampling by factor of 8
plt.figure(figsize=(14, 10))
plt.imshow(im[::ds, ::ds])

In [ ]:
#img = im[::ds, ::ds].astype(np.uint8).copy()
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # convert to greyscale for edge detection
print(gray.shape)

In [ ]:
#from skimage.feature import corner_harris, corner_peaks, corner_subpix
#coords = corner_peaks(corner_harris(gray), min_distance=50)
#coords_subpix = corner_subpix(gray, coords, window_size=13)
#print(coords_subpix)
#coords
#plt.figure(figsize=(8,8))
#plt.imshow(gray)
#plt.scatter(coords[:, 0], coords[:, 1], c='r')

In [ ]:
cv2.Canny?

In [ ]:
#print(lines[0, 0, 0]) 
#print(lines[0, 0, 1] * 180 / np.pi) 

In [ ]:
# indices, `DP' = data point
DP_1 = 0
DP_2 = 1
X_COORD = 0
Y_COORD = 1

In [ ]:
cv2.HoughLinesP?

In [ ]:
linesP.shape

In [ ]:
'''
@param threshold Accumulator threshold parameter. Only those lines are returned that get enough
       votes ( \f$>\texttt{threshold}\f$ ).
@param minLineLength Minimum line length. Line segments shorter than that are rejected.
@param maxLineGap Maximum allowed gap between points on the same line to link them.
'''
#img = im[::ds, ::ds].astype(np.uint8).copy()
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
img_h = img.shape[0]
img_w = img.shape[1]
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

#edges = cv2.Canny(gray, 100, 500, L2gradient=True)
edges = cv2.Canny(gray, 20, 400, L2gradient=True) # GLN_3447

# @param rho Distance resolution of the accumulator in pixels.
rho = 1  
# @param theta Angle resolution of the accumulator in radians.
theta = np.pi / 90  # note, with 180 getting too many overlapping lines.

lines  =  cv2.HoughLines(edges, rho, theta, threshold=100)
#linesP = cv2.HoughLinesP(edges, 1, theta, 110)
#linesP = cv2.HoughLinesP(edges, rho, theta, threshold=100, minLineLength=200, maxLineGap=100)

#print(lines.shape)  # (N - number of lines found, r - pixels, theta - radians)
#coords = []
N = 1

coords = np.zeros((N, 2, 2)).astype('int') # points, start/end, x/y
angles = np.zeros(N)
for i in range(len(lines[:N])):
    for r, theta in lines[i]:
        # theta is normal to the line wrt x-axis, so we subtract 90
        if r > 0:
            angles[i] = 90 - theta * 180 / np.pi
        else: 
            angles[i] = 270 - theta * 180 / np.pi
        print(i, '%.2f' % angles[i])
        a = np.cos(theta)
        b = np.sin(theta)
        x0, y0 = a * r, b * r
        x1 = int(x0 + img_w * -b)
        y1 = int(y0 + img_w *  a)
        x2 = int(x0 - img_w * -b)
        y2 = int(y0 - img_w *  a)
        #coords.append([[x1, y1], [x2, y2]])
        coords[i, DP_1, X_COORD] = x1
        coords[i, DP_1, Y_COORD] = y1
        coords[i, DP_2, X_COORD] = x2
        coords[i, DP_2, Y_COORD] = y2
        print(i, (x1, y1), (x2, y2))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
plt.figure(figsize=(8,8))
plt.imshow(img)
plt.xlabel('x')
plt.ylabel('y')
plt.show()        

In [ ]:
coords

## Post processing

In [ ]:
tol = 5  # tolerance in degrees
ref = angles[0]
candidates = []
candidates.append((0, a))
similar_angles = []
for i, a in enumerate(angles[1:]):
    if a >= ref and ref + tol > a:
        similar_angles.append((i + 1, a))
    else:
        candidates.append((i + 1, a))
print(similar_angles)
print(candidates)

# Of the lines with a similar angle, keep only those that are offset
ref_coord = coords[0]

for i, a in similar_angles:
    if abs(coords[i, DP_1, X_COORD] - ref_coord[DP_1, X_COORD]) > 50:
        print('x-coords differ')
        candidates.append((i, a))
        ref_coord = coords[i]  # update ref_coord
        
    if abs(coords[i, DP_1, Y_COORD] - ref_coord[DP_1, Y_COORD]) > 50:
        print('y-coords differ')
        candidates.append((i, a))
        ref_coord = coords[i]  # update ref_coord

print(candidates)

plt.figure(figsize=(8,8))
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
for (i, _) in candidates:
    cv2.line(img, (coords[i, DP_1, X_COORD], coords[i, DP_1, Y_COORD]), 
                  (coords[i, DP_2, X_COORD], coords[i, DP_2, Y_COORD]), (0, 0, 255), 2)
plt.imshow(img)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Probabilistic HoughLines

In [ ]:
#edges = cv2.Canny(gray, 20, 400, L2gradient=True)
plt.figure(figsize=(12, 10))
plt.imshow(edges)

In [ ]:
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
img_h = img.shape[0]
img_w = img.shape[1]
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# @param canny_threshold1
# @param canny_threshold2
canny_thresh1 = 20
canny_thresh2 = 400

# first pass using HoughLines to find rotation angle
'''
edges = cv2.Canny(gray, threshold1=canny_thresh1, threshold2=canny_thresh2, L2gradient=True) # GLN_3447
lines = cv2.HoughLines(edges, rho, theta, threshold=100)
rot_deg = 90 - lines[0][0][1] * rad2deg  # lines are sorted by number of votes, 0th line is best
imgr = transform.rotate(np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8), -rot_deg, mode='symmetric')
imgr = (255 * imgr).astype(np.uint8)
grayr = cv2.cvtColor(imgr, cv2.COLOR_RGB2GRAY)
'''
# run the Canny edge detector on the rotated gray scale image
edges = cv2.Canny(gray, threshold1=canny_thresh1, threshold2=canny_thresh2, L2gradient=True) # GLN_3447

# @param rho Distance resolution of the accumulator (pixels).
rho = 1  

# @param theta Angle resolution of the accumulator (radians).
theta = np.pi / 90

# @param threshold Accumulator threshold, return lines with more than threshold of votes. (intersection points)
threshold = 100

# @param minLineLength Minimum line length. Line segments shorter than that are rejected. (pixels)
mLL = 200

# @param maxLineGap Maximum allowed gap between points on the same line to link them. (pixels)
mLG = 100

# run the probabilistic hough lines transform
linesP = cv2.HoughLinesP(edges, rho, theta, threshold=threshold, minLineLength=mLL, maxLineGap=mLG)

# @param lines The extremes of the detected lines if any (<N_LINES_FOUND>, 1, x_0, y_0, x_1, y_1). (pixels)
print('Found %d lines' % len(linesP))

N = 20  # top N results to draw
if linesP is not None:
    for i in range(len(linesP[:N])):
        l = linesP[i][0]
        cv2.line(img, (l[0], l[1]), (l[2], l[3]), (255,0,0), 3, cv2.LINE_AA)
plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# line below origin, rho positive, angle less than 180
# line above origin, rho negative, angle less than 180
lines = cv2.HoughLines(edges, rho, theta, threshold=100)
rot_deg = 90 - lines[0][0][1] * rad2deg
t = rot_deg / rad2deg

In [ ]:
plt.figure(figsize=(12, 10))
center = (coordsP[0, DP_1, X_COORD], coordsP[0, DP_1, Y_COORD])
imgr = transform.rotate(np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8), -rot_deg, center=center, resize=False)
imgr = (255 * imgr).astype(np.uint8)
plt.imshow(imgr[:, :])
plt.show()

In [ ]:
coordsP = np.zeros((np.minimum(N, len(linesP)), 2, 2)).astype('int') # points, start/end, x/y

for i in range(len(linesP[:N])):
    l = linesP[i][0]
    coordsP[i, DP_1, X_COORD] = l[0] # x1
    coordsP[i, DP_1, Y_COORD] = l[1] # y1
    coordsP[i, DP_2, X_COORD] = l[2] # x2
    coordsP[i, DP_2, Y_COORD] = l[3] # y2
    
R = np.array([[np.cos(-t), -np.sin(-t)],
              [np.sin(-t),  np.cos(-t)]])
coordsP_r = np.dot(coordsP, R)
coordsP_r[0]

In [ ]:
for i in range(len(linesP[:N])):
    
    #l = linesP[i][0]
    #cv2.line(img, (l[0], l[1]), (l[2], l[3]), (255, 0, 0), 3, cv2.LINE_AA)
    '''
    cv2.line(imgr, (int(coordsP_r[i, DP_1, X_COORD]),
                    int(coordsP_r[i, DP_1, Y_COORD])),
                   (int(coordsP_r[i, DP_2, X_COORD]),
                    int(coordsP_r[i, DP_2, Y_COORD])),
             (128, 0, 255), 3, cv2.LINE_AA)
    '''
    cv2.line(imgr, (int(coordsP_r[i, DP_1, X_COORD]),
                    int(coordsP_r[i, DP_1, Y_COORD] + (coordsP[0, DP_1, Y_COORD] - coordsP_r[0, DP_1, Y_COORD]))),
                   (int(coordsP_r[i, DP_2, X_COORD]),
                    int(coordsP_r[i, DP_2, Y_COORD] + (coordsP[0, DP_1, Y_COORD] - coordsP_r[0, DP_1, Y_COORD]))),
             (128, 0, 255), 3, cv2.LINE_AA)         

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(imgr)

# Find intersection points